In [1]:
from glob import glob
import os
from PIL import Image
import rasterio as rio
import numpy as np

In [34]:
tif_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2/Polygon_Crops/MSK/Individual_Polygons/TIF"
tif_list = glob(tif_dir + "/*.tif",recursive=True)
tif_files = [i.split("/")[-1].split(".")[0] + ".jpg" for i in tif_list]
out_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/Polygon_Crops/MSK/Individual_Polygons/JPG/"

for index,tif in enumerate(tif_list):
    im = Image.open(tif)
#     im.thumbnail(im.size)
#     im.save(out_dir + tif_files[index], "JPEG", quality=100)

# Updated Code

# Convert All Rasters to UINT8

In [122]:
def add_trailing_slash(path):
    if path[-1] != '/':
        path += '/'
    return path

In [123]:
def create_dir(output_dir):
    # If the output folder doesn't exist, create it
    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)

In [150]:
def convert_to_int8(input_dir,output_dir):
    
    input_dir = add_trailing_slash(input_dir)
    output_dir = add_trailing_slash(output_dir)
    
    create_dir(output_dir)
    
    file_list = glob(input_dir + "/*/*.tif")
   
    
    for image_path in file_list:
        new_dir = output_dir + image_path.split("/")[-2]
        new_dir = add_trailing_slash(new_dir)
        create_dir(new_dir)
        
        file_path_append = ("/").join(image_path.split("/")[-2:]) 
        
        with rio.open(image_path, 'r') as r:
            rast = r.read([6,7,8])
            rast = rast.astype(np.uint8)
            r_meta = r.meta
            r_meta["count"] = 3
            r_meta["dtype"] = 'uint8'


        with rio.open(output_dir + file_path_append, 'w', **r_meta) as outf:
            outf.write(rast)
    

In [151]:
input_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v4/LabelBox_Labelling/TIF/"
output_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v4/LabelBox_Labelling/TIF_converted"

file_list = convert_to_int8(input_dir,output_dir)


# Convert All TIFs to JPEG - Preserves Directory Structure

In [165]:
def convert_to_jpg(input_dir,output_dir):
    
    input_dir = add_trailing_slash(input_dir)
    output_dir = add_trailing_slash(output_dir)
    
    create_dir(output_dir)
    
    file_list = glob(input_dir + "/*/*.tif") 
    
    
    for image_path in file_list:
        
        new_dir = output_dir + image_path.split("/")[-2]
        
        new_dir = add_trailing_slash(new_dir)
        
        create_dir(new_dir)
        
        file_name = image_path.split("/")[-1].split(".")[-2]
        
        im = Image.open(image_path)
        im.thumbnail(im.size)
        im.save(new_dir + file_name + ".jpg", "JPEG", quality=100)


In [166]:
input_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v4/LabelBox_Labelling/TIF_converted"
output_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v4/LabelBox_Labelling/JPG/"

convert_to_jpg(input_dir,output_dir)
